In [ ]:
import os
arch = os.getenv("ARGS", "real")

In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import petsc4py
    import slepc4py
else:
    try:
        import slepc4py
    except ImportError:
        if arch != "complex":
            !wget "https://fem-on-colab.github.io/releases/slepc4py-install-real.sh" -O "/tmp/slepc4py-install.sh" && bash "/tmp/slepc4py-install.sh"
        else:
            !wget "https://fem-on-colab.github.io/releases/slepc4py-install-complex.sh" -O "/tmp/slepc4py-install.sh" && bash "/tmp/slepc4py-install.sh"
        import petsc4py
        import slepc4py

In [ ]:
petsc4py.init()
from petsc4py import PETSc

In [ ]:
import numpy as np
if arch != "complex":
    assert not np.issubdtype(PETSc.ScalarType, np.complexfloating)
else:
    assert np.issubdtype(PETSc.ScalarType, np.complexfloating)

In [ ]:
slepc4py.init()
from slepc4py import SLEPc

In [ ]:
A = PETSc.Mat().createAIJ([3, 3], nnz=1)
for i in range(3):
    A.setValue(i, i, i + 1)
A.assemble()

In [ ]:
assert np.allclose(A.getValues(range(3), range(3)), np.diag([1., 2., 3.]))

In [ ]:
eps = SLEPc.EPS().create()
eps.setOperators(A)
eps.setProblemType(SLEPc.EPS.ProblemType.HEP)
eps.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL)
eps.solve()
assert eps.getConverged() == 3
for i in range(3):
    eig_i = eps.getEigenvalue(i)
    assert np.isclose(eig_i.real, i + 1)
    assert np.isclose(eig_i.imag, 0)

In [ ]:
for package in ("mumps", "superlu", "superlu_dist"):
    eps = SLEPc.EPS().create()
    eps.setOperators(A)
    eps.setProblemType(SLEPc.EPS.ProblemType.HEP)
    eps.setWhichEigenpairs(SLEPc.EPS.Which.TARGET_REAL)
    eps.setTarget(0)
    st = eps.getST()
    st.setType(SLEPc.ST.Type.SINVERT)
    st.setShift(0)
    ksp = st.getKSP()
    ksp.setType("preonly")
    pc = ksp.getPC()
    pc.setType("lu")
    pc.setFactorSolverType(package)
    eps.solve()
    assert eps.getConverged() == 3
    for i in range(3):
        eig_i = eps.getEigenvalue(i)
        assert np.isclose(eig_i.real, i + 1)
        assert np.isclose(eig_i.imag, 0)